In [1]:
import sys 
import os
lib_path = './lib'
sys.path.insert(0, lib_path)     ## add lib path to sys path 
import pandas as pd
import MultiContagion as mc
import igraph
import random
import numpy as np
from matplotlib import pylab as plt
import scipy.stats
import copy
from scipy.sparse import linalg as LA
#import powerlaw
%matplotlib inline

In [2]:
## specify the year you want to run 
year = 2016

In [3]:
import_path = "./data/adjacency_matrix/"
export_path = './results/centralities/'

In [4]:
#import the aggregate adjacency matrix
aggregate_am = np.genfromtxt (import_path + 'AM4_all_nodes_aggregate'+str(year)+'.csv', delimiter=",")
df_names = pd.read_csv(import_path+ 'all_country_name4.csv', header=None)
names = list(df_names[0])
Aggregate_g = igraph.Graph.Weighted_Adjacency(list(aggregate_am))
Aggregate_g.vs["name"] = copy.deepcopy(names)

In [5]:
#Import the adjacency matrix and make graphs of all the layers.
cdis_equity_am = np.genfromtxt (import_path +'AM4_all_nodesCDIS-equity'+str(year)+'.csv', delimiter=",")
cdis_equity_g = igraph.Graph.Weighted_Adjacency(list(cdis_equity_am))
cdis_equity_g.vs["name"] = copy.deepcopy(names)

cdis_debt_am = np.genfromtxt (import_path +'AM4_all_nodesCDIS-debt'+str(year)+'.csv', delimiter=",")
cdis_debt_g = igraph.Graph.Weighted_Adjacency(list(cdis_debt_am))
cdis_debt_g.vs["name"] = copy.deepcopy(names)

cpis_equity_am = np.genfromtxt (import_path +'AM4_all_nodesCPIS-equity'+str(year)+'.csv', delimiter=",")
cpis_equity_g = igraph.Graph.Weighted_Adjacency(list(cpis_equity_am))
cpis_equity_g.vs["name"] = copy.deepcopy(names)

cpis_debt_am = np.genfromtxt (import_path +'AM4_all_nodesCPIS-debt'+str(year)+'.csv', delimiter=",")
cpis_debt_g = igraph.Graph.Weighted_Adjacency(list(cpis_debt_am))
cpis_debt_g.vs["name"] = copy.deepcopy(names)

bis_am = np.genfromtxt (import_path +'AM4_all_nodesBIS'+str(year)+'.csv', delimiter=",")
bis_g = igraph.Graph.Weighted_Adjacency(list(bis_am))
bis_g.vs["name"] = copy.deepcopy(names)

## Data check

In [6]:
cdis_debt_g.vs["name"].index("United States"), cdis_debt_g.vs["name"].index("China  P.R.: Mainland")

(201, 43)

In [7]:
print ("CDIS eq = ", cdis_equity_am[201][43])
print ("CDIS dt = ", cdis_debt_am[201][43] )
print ("CPIS eq = ",cpis_equity_am[201][43])
print ("CPIS dt = ",cpis_debt_am[201][43] )
print ("BIS = ", bis_am[201][43])

CDIS eq =  19712.0
CDIS dt =  19344.0
CPIS eq =  177738.0
CPIS dt =  1451834.0
BIS =  100432.0


In [8]:
cdis_debt_am[201][43]

19344.0

## Lists definitions

In [9]:
G_list = [cdis_equity_g, cdis_debt_g, cpis_equity_g, cpis_debt_g, bis_g]

In [10]:
AM_list = [cdis_equity_am, cdis_debt_am, cpis_equity_am, cpis_debt_am, bis_am ]

In [11]:
In_list = [ np.array(g.strength(mode ="IN", weights= g.es["weight"])) for g in G_list]
Out_list = [ np.array(g.strength(mode ="OUT", weights= g.es["weight"])) for g in G_list]

In [12]:
def make_supra_coup(list_adj, list_vector):
    "Takes a list of adjacency matrix and list of vectors to use as coupling for layers"
    n_layers = len(list_adj)
    dim = list_adj[0].shape[0]
    sup_list = []
    for i in range(n_layers):
        row = []
        for j in range(n_layers):
            if i == j:
                row.append(list_adj[i])
            else:
                row.append(np.diag(list_vector[j]))
        sup_list.append(row)
    return np.bmat(sup_list)

## Centrality calculation

In [13]:
supra_adj = make_supra_coup(AM_list, In_list)
G_supra = igraph.Graph.Weighted_Adjacency(supra_adj.tolist())
complete_multi_pr = G_supra.personalized_pagerank(weights = G_supra.es["weight"])

In [14]:
G_supra = igraph.Graph.Weighted_Adjacency(supra_adj.tolist())

In [15]:
#G_supra = igraph.Graph.Weighted_Adjacency(supra_adj.tolist())

In [16]:
G_supra.is_weighted(), G_supra.is_directed()

(True, True)

In [17]:
def save_centrality(y, namefile, namecent , country, cent):
    f = open(export_path+namefile+str(y) +".csv", "w")
    f.write("Country,"+namecent+"\n")
    n = len(country)
    for i in range(n):
        f.write(str(country[i]) + "," + str(cent[i])+ "\n")
    f.close()

## PageRank

In [18]:
pr_agg = Aggregate_g.personalized_pagerank(weights = Aggregate_g.es["weight"], damping = 0.85)
node_names = np.array(copy.deepcopy(names))
inds = np.array(pr_agg).argsort()[::-1][:]
sort_names = node_names[inds]
pr_agg_ord = np.array(pr_agg)[inds]

In [19]:
sort_names[:30]

array(['United States', 'United Kingdom', 'Luxembourg', 'Netherlands',
       'France', 'Germany', 'Japan', 'Switzerland', 'Cayman Islands',
       'China  P.R.: Hong Kong', 'China  P.R.: Mainland', 'Canada',
       'Italy', 'Belgium', 'Singapore', 'Bermuda', 'Sweden', 'Norway',
       'Ireland', 'Spain', 'Australia', 'Denmark', 'Korea  Republic of',
       'Austria', 'Russian Federation', 'South Africa', 'Mauritius',
       'Finland', 'Brazil', 'Jersey'], dtype='<U36')

In [20]:
save_centrality(year, "Agg_PageRank", "PageRank", sort_names, pr_agg_ord)

In [21]:
def get_pagerank_list(G, n_layers = 5, names= names, r = 0.85):
    pr = G.personalized_pagerank(weights = G.es["weight"], damping = r)
    multi_pr = []
    n_nodes = int((len(G.degree()))/5)
    multi_pr = []
    for i in range(n_nodes):
        cent = 0
        for k in range(n_layers):
            cent += pr[i + n_nodes*k] 
        multi_pr.append(cent)
    node_names = np.array(copy.deepcopy(names))
    inds = np.array(multi_pr).argsort()[::-1][:]
    sort_names_multi_pr = node_names[inds]
    multi_sort_pr = np.array(multi_pr)[inds]
    return sort_names_multi_pr, multi_sort_pr

In [22]:
n_pr, num_pr = get_pagerank_list(G_supra, r = 0.85)

In [23]:
n_pr, num_pr = get_pagerank_list(G_supra, r = 0.85)
n_pr[:30]

array(['United States', 'France', 'United Kingdom', 'Luxembourg',
       'Germany', 'Netherlands', 'Japan', 'Switzerland',
       'China  P.R.: Hong Kong', 'Canada', 'Cayman Islands', 'Italy',
       'Belgium', 'China  P.R.: Mainland', 'Austria',
       'Korea  Republic of', 'Norway', 'Singapore', 'Spain', 'Sweden',
       'Curacao & St. Maarten', 'South Africa', 'Denmark', 'Australia',
       'Bermuda', 'Russian Federation', 'Taiwan Province of China',
       'Mauritius', 'Saudi Arabia', 'Ireland'], dtype='<U36')

In [24]:
def make_supra_coup2(list_adj, list_vector):
    "Takes a list of adjacency matrix and list of vectors to use as coupling for layers"
    n_layers = len(list_adj)
    dim = list_adj[0].shape[0]
    sup_list = []
    for i in range(n_layers):
        row = []
        for j in range(n_layers):
            if i == j:
                row.append(list_adj[i])
            else:
                row.append(np.diag(list_vector[i]))
        sup_list.append(row)
    return np.bmat(sup_list)

In [25]:
supra_adj2 = make_supra_coup2(AM_list, In_list)
G_supra2 = igraph.Graph.Weighted_Adjacency(supra_adj2.tolist())

In [26]:
n_pr2, num_pr2 = get_pagerank_list(G_supra2)

In [27]:
n_pr2;

In [28]:
n_pr2, num_pr2 = get_pagerank_list(G_supra2, r = 0.85)
n_pr2[:30]

array(['United States', 'Luxembourg', 'United Kingdom', 'Netherlands',
       'France', 'Germany', 'Japan', 'Switzerland',
       'China  P.R.: Hong Kong', 'Italy', 'Cayman Islands', 'Belgium',
       'Canada', 'Mauritius', 'Denmark', 'China  P.R.: Mainland',
       'Singapore', 'Kuwait', 'Korea  Republic of', 'Norway', 'Sweden',
       'Austria', 'Bermuda', 'Spain', 'South Africa',
       'Russian Federation', 'Australia', 'Guernsey', 'Thailand',
       'Saudi Arabia'], dtype='<U36')

In [29]:
save_centrality(year, "MultCoupCashFlow_PageRank", "PageRank centrality", n_pr, num_pr)

In [30]:
save_centrality(year, "MultCoupRiskFlow_PageRank", "PageRank centrality", n_pr2, num_pr2)

## Eigenvector

In [31]:
def get_eigenvector_list(G, n_layers = 5, names= names):
    pr = G.eigenvector_centrality(weights = G.es["weight"])
    multi_pr = []
    n_nodes = int((len(G.degree()))/5)
    multi_pr = []
    for i in range(n_nodes):
        cent = 0
        for k in range(n_layers):
            cent += pr[i + n_nodes*k] 
        multi_pr.append(cent)
    node_names = np.array(copy.deepcopy(names))
    inds = np.array(multi_pr).argsort()[::-1][:]
    sort_names_multi_pr = node_names[inds]
    multi_sort_pr = np.array(multi_pr)[inds]
    return sort_names_multi_pr, multi_sort_pr

In [32]:
n_ei, num_ei = get_eigenvector_list(G_supra)
n_ei2, num_ei2 = get_eigenvector_list(G_supra2)

D:\data\chengyu\_GFF\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: Weighted directed graph in eigenvector centrality at src\centrality.c:362
  


In [33]:
n_ei2;

In [34]:
save_centrality(year, "MultCoupCashFlow_Eigenvector", "Eigenvector centrality", n_ei, num_ei)
save_centrality(year, "MultCoupRiskFlow_Eigenvector", "Eigenvector centrality", n_ei2, num_ei2)

## Hubs and Auth

In [35]:
def G_list_hub(G_adj_list):
    G_hub = []
    for g in G_adj_list:
        new = np.dot(g, g.transpose())
        G_hub.append(new)   
    return G_hub

In [36]:
def G_list_auth(G_adj_list):
    G_hub = []
    for g in G_adj_list:
        new = np.dot( g.transpose(), g)
        G_hub.append(new)   
    return G_hub

In [37]:
def hub_auth_diag(Coup_list):
    coup = []
    for c in Coup_list:
        coup.append(c*c)
    return coup

In [38]:
G_supra_hub = make_supra_coup(G_list_hub(AM_list), hub_auth_diag(In_list))
G_supra_auth = make_supra_coup(G_list_auth(AM_list), hub_auth_diag(In_list))

In [39]:
G_supra_hub[0].shape

(1, 1065)

In [40]:
def get_hub_list(AM, n_layers = 5, names= names):
    hub_eig = LA.eigs(AM, k=1, which='LM')[1]
    multi_pr = []
    #print hub_eig[0]
    n_nodes = int((AM[0].shape[1])/5.)
    multi_pr = []
    for i in range(n_nodes):
        cent = 0
        for k in range(n_layers):
            
            cent += abs(hub_eig[i + n_nodes*k])
        
        multi_pr.append(cent[0])
    
    node_names = np.array(copy.deepcopy(names))
    inds = np.array(multi_pr).argsort()[::-1][:]
    sort_names_multi_pr = node_names[inds]
    multi_sort_pr = np.array(multi_pr)[inds]
    return sort_names_multi_pr, multi_sort_pr

In [41]:
n_hub, num_hub = get_hub_list(G_supra_hub)
n_auth, num_auth = get_hub_list(G_supra_auth)

In [42]:
n_auth, num_auth = get_hub_list(G_supra_auth)

In [43]:
n_auth[:20]

array(['United States', 'Luxembourg', 'Netherlands', 'United Kingdom',
       'Germany', 'Japan', 'France', 'China  P.R.: Hong Kong',
       'Switzerland', 'Bermuda', 'Cayman Islands', 'Belgium', 'Canada',
       'Italy', 'Ireland', 'Norway', 'Jersey', 'Singapore',
       'China  P.R.: Mainland', 'Australia'], dtype='<U36')

In [44]:
n_hub[:20]

array(['United States', 'Netherlands', 'United Kingdom', 'France',
       'Luxembourg', 'Germany', 'Cayman Islands', 'Ireland',
       'Switzerland', 'Japan', 'Italy', 'Canada', 'Belgium', 'Australia',
       'Spain', 'China  P.R.: Hong Kong', 'Brazil',
       'China  P.R.: Mainland', 'Sweden', 'Singapore'], dtype='<U36')

In [45]:
#save_centrality(year, "MultCoupCashFlow_Hub", "Hub Score", n_hub, num_hub)
#save_centrality(year, "MultCoupCashFlow_Auth", "Authority Score", n_auth, num_auth)

### other coup

In [46]:
G_supra_hub2 = make_supra_coup2(G_list_hub(AM_list), hub_auth_diag(In_list))
G_supra_auth2 = make_supra_coup2(G_list_auth(AM_list), hub_auth_diag(In_list))

In [47]:
n_hub2, num_hub2 = get_hub_list(G_supra_hub2)
n_auth2, num_auth2 = get_hub_list(G_supra_auth2)

In [48]:
#n_hub2, num_hub2 = get_eigenvector_list(igraph.Graph.Weighted_Adjacency(G_supra_hub2.tolist()))
#n_auth2, num_auth2 = get_eigenvector_list(igraph.Graph.Weighted_Adjacency(G_supra_auth2.tolist()))

In [49]:
n_auth2[:30]

array(['United States', 'Luxembourg', 'Netherlands', 'Japan',
       'United Kingdom', 'Germany', 'China  P.R.: Hong Kong', 'France',
       'Switzerland', 'Bermuda', 'Canada', 'Belgium', 'Italy',
       'Cayman Islands', 'Norway', 'Ireland', 'Australia', 'Singapore',
       'Sweden', 'Jersey', 'Spain', 'China  P.R.: Mainland',
       'South Africa', 'Guernsey', 'Denmark', 'Gibraltar', 'Finland',
       'Austria', 'Brazil', 'Korea  Republic of'], dtype='<U36')

In [50]:
n_hub2[:30]

array(['United States', 'United Kingdom', 'Netherlands', 'Luxembourg',
       'Ireland', 'France', 'Cayman Islands', 'Germany', 'Switzerland',
       'Japan', 'Belgium', 'Canada', 'Italy', 'Australia',
       'China  P.R.: Mainland', 'Spain', 'Bermuda',
       'China  P.R.: Hong Kong', 'Brazil', 'Singapore', 'Sweden',
       'Korea  Republic of', 'Jersey', 'India', 'Mexico', 'Austria',
       'Denmark', 'Taiwan Province of China', 'Russian Federation',
       'Gibraltar'], dtype='<U36')

In [51]:
save_centrality(year, "MultCoupRiskFlow_Hub", "Hub Score", n_hub2, num_hub2)
save_centrality(year, "MultCoupRiskFlow_Auth", "Authority Score", n_auth2, num_auth2)

In [52]:
num_hub2 [:20]

array([1.76286170e+00, 1.84199019e-02, 1.33670803e-02, 1.02602922e-02,
       8.43572103e-03, 7.82565918e-03, 7.10830423e-03, 6.67439822e-03,
       6.36681691e-03, 5.22431936e-03, 3.25415963e-03, 3.09434344e-03,
       2.98235368e-03, 2.46083190e-03, 2.44806379e-03, 2.31451529e-03,
       2.25062805e-03, 2.21403131e-03, 1.89574932e-03, 1.74600431e-03])